In [27]:
"""
Our company is not working efficiently. We spend way too much time drafting documents and this needs to be fixed!


For the company, you need to create an AI Agentic System that can speed up drafting documents, emails, etc.
The AI Agentic System should have human-AI collaboration meaning the Human should be able to provide continuous feedback and
the AI agent should stop when the Human is happy with the draft.
The system should be fast and be able to save the drafts.

"""

'\nOur company is not working efficiently. We spend way too much time drafting documents and this needs to be fixed!\n\n\nFor the company, you need to create an AI Agentic System that can speed up drafting documents, emails, etc. \nThe AI Agentic System should have human-AI collaboration meaning the Human should be able to provide continuous feedback and \nthe AI agent should stop when the Human is happy with the draft. \nThe system should be fast and be able to save the drafts.\n\n'

In [ ]:
!pip install langchain_anthropic

In [29]:
# We want to use LangGraph and structure:
# start -> agent -> tools -> continue -> agent
# start -> agent -> tools -> end
# However, React agent: goes back to the agent


In [30]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, ToolMessage, SystemMessage
from langchain.tools import tool
from langchain_anthropic import ChatAnthropic
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

In [31]:
from getpass import getpass
import os

claude_api_key = getpass("Enter your Claude API Key: ")
os.environ["ANTHROPIC_API_KEY"] = claude_api_key

Enter your Claude API Key: ··········


In [32]:
# This is the global variable to store the document content
document_content = ""

In [33]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]

In [34]:
@tool
def update(content: str) -> str:
  """ Updates the document with the provided content. """
  global document_content
  document_content = content
  return f"Document has been updated successfully! The current content is: \n{document_content}"

In [35]:
@tool
def save(filename: str) -> str:
  """ Saves the current document to a text file and finish the process.

  Args: filename: Name for the text file.
  """
  global document_content

  if not filename.endswith('.txt'):
    filename = f"{filename}.txt"

  try:

    with open(filename, 'w') as file:
      file.write(document_content)
    print(f"\n Document has been saved to: {filename}")
    return f"Document has been saved succeesfully to '{filename}'."

  except Exception as e:
    return f"Error saving document: {str(e)}"

In [36]:
tools = [update, save]

In [37]:
model = ChatAnthropic(
    model = "claude-sonnet-4-20250514",
    anthropic_api_key = claude_api_key
).bind_tools(tools)

In [38]:
def our_agent(state: AgentState) -> AgentState:
  system_prompt = SystemMessage(content = f"""
  You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.

  - If the user wants to update or modify content, use the 'update' tool with the complete updated document.
  - If the user wants to save and finish, you need to use the 'save' tool.
  - Make sure to always show the current document state after modifications.

  The current document content is: {document_content}

  """)

  if not state['messages']:
    user_input = "I'm ready to help you update a document. What would you like to create?"
    user_message = HumanMessage(content = user_input)

  else:
    user_input = input("\n What would you like to do with the document?")
    print(f"\n USER: {user_input}")
    user_message = HumanMessage(content = user_input)


  all_messages = [system_prompt] + list(state["messages"]) + [user_message]

  response = model.invoke(all_messages)

  print(f"\n AI: {response.content}")
  if hasattr(response, "tool_calls") and response.tool_calls:
    print(f"USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

  return {'messages': list(state['messages']) + [user_message, response]}

In [39]:
def should_continue(state: AgentState) -> str:
  """ Determine if we should continue or end the conversation. """
  messages = state['messages']

  if not messages:
    return "continue"

  # This looks for the most recent end message...
  for message in reversed(messages):
    # ... and checks if this is a ToolMessage resulting from save
    if (isinstance(message, ToolMessage) and
        "saved" in message.content.lower() and
        "document" in message.content.lower()):
        return "end" # goes to the end edge which leads to the end point

  return "continue"

In [40]:
def print_messages(messages):
  """ Function I made to print the messages in a more readable format"""

  if not messages:
    return

  for message in messages[-3:]:
    if isinstance(message, ToolMessage):
      print(f"\n TOOL RESULT: {message.content}")

In [41]:
graph = StateGraph(AgentState)
graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))
graph.set_entry_point("agent")
graph.add_edge("agent", "tools")
graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        # Edge: Node
        "continue": "agent",
        "end": END
    },
)

app = graph.compile()

In [42]:
def run_document():
  print(f"\n ====== DRAFTER ======")

  state = {"messages": []}

  for step in app.stream(state, stream_mode="values"):
    if "messages" in step:
      print_messages(step["messages"])

  print(f"\n ====== DRAFTER FINISHED ======")

In [43]:
run_document()


 ====== DRAFTER ======

 AI: I'm ready to help you create and update a document! The current document is empty, so we can start fresh with whatever you'd like to write.

What would you like to create? For example:
- A letter or email
- A report or article
- A story or creative writing
- Meeting notes or documentation
- A list or outline
- Something else entirely

Just let me know what content you'd like to add, and I'll help you build and refine your document!

 What would you like to do with the document?write an email to Tom that I will not be able to make it for the meeting

 USER: write an email to Tom that I will not be able to make it for the meeting

 AI: [{'text': "I'll help you write an email to Tom about not being able to make the meeting. Let me create that for you:", 'type': 'text'}, {'id': 'toolu_01CDyf2PNWDtVzqWsNXHburt', 'input': {'content': "Subject: Unable to Attend Meeting\n\nHi Tom,\n\nI hope this email finds you well. I'm writing to let you know that I will not be 